# 在Dynex平台上使用Python进行计算BQM (3)

Dynex是全球首个基于DynexSolve芯片算法的神经形态超级计算区块链，采用了一种“有用工作证明”（PoUW）方法来解决现实世界问题。这个示例演示了如何使用Dynex SDK来使用Python在Dynex平台上进行计算。

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import dynex
import dimod

## 构建一个二次二元模型

二次二元模型（BQM）是以下形式的问题:

<math xmlns="http://www.w3.org/1998/Math/MathML" display="block">
  <mi>E</mi>
  <mo stretchy="false">(</mo>
  <mrow data-mjx-texclass="ORD">
    <mi mathvariant="bold">v</mi>
  </mrow>
  <mo mathvariant="bold" stretchy="false">)</mo>
  <mo mathvariant="bold">=</mo>
  <munder>
    <mo data-mjx-texclass="OP">&#x2211;</mo>
    <mrow data-mjx-texclass="ORD">
      <mi mathvariant="bold">i</mi>
    </mrow>
  </munder>
  <msub>
    <mi mathvariant="bold">a</mi>
    <mi mathvariant="bold">i</mi>
  </msub>
  <msub>
    <mi mathvariant="bold">v</mi>
    <mi mathvariant="bold">i</mi>
  </msub>
  <mo mathvariant="bold">+</mo>
  <munder>
    <mo data-mjx-texclass="OP">&#x2211;</mo>
    <mrow data-mjx-texclass="ORD">
      <mi mathvariant="bold">i</mi>
      <mo mathvariant="bold">&lt;</mo>
      <mi mathvariant="bold">j</mi>
    </mrow>
  </munder>
  <msub>
    <mi mathvariant="bold">b</mi>
    <mrow data-mjx-texclass="ORD">
      <mi mathvariant="bold">i</mi>
      <mo mathvariant="bold">,</mo>
      <mi mathvariant="bold">j</mi>
    </mrow>
  </msub>
  <msub>
    <mi mathvariant="bold">v</mi>
    <mi mathvariant="bold">i</mi>
  </msub>
  <msub>
    <mi mathvariant="bold">v</mi>
    <mi mathvariant="bold">j</mi>
  </msub>
  <mo mathvariant="bold">+</mo>
  <mi mathvariant="bold">c</mi>
  <mstyle scriptlevel="0">
    <mspace width="2em"></mspace>
  </mstyle>
  <mstyle scriptlevel="0">
    <mspace width="2em"></mspace>
  </mstyle>
  <msub>
    <mi mathvariant="bold">v</mi>
    <mi mathvariant="bold">i</mi>
  </msub>
  <mo>&#x2208;</mo>
  <mo fence="false" stretchy="false">{</mo>
  <mo mathvariant="bold">&#x2212;</mo>
  <mn mathvariant="bold">1</mn>
  <mo mathvariant="bold">,</mo>
  <mo mathvariant="bold">+</mo>
  <mn mathvariant="bold">1</mn>
  <mo fence="false" stretchy="false">}</mo>
  <mtext mathvariant="bold">&#xA0;or&#xA0;</mtext>
  <mo fence="false" stretchy="false">{</mo>
  <mn mathvariant="bold">0</mn>
  <mo mathvariant="bold">,</mo>
  <mn mathvariant="bold">1</mn>
  <mo fence="false" stretchy="false">}</mo>
</math>

其中abc是实数。

我们需要定义一个代表我们问题的二次模型（QM）。构建一个二次二元模型（BQM）的最简单方式是使用dimod的符号变量。对于BQM中的每个数学变量，我们使用dimod定义一个符号二元变量。一旦QM被定义，它将被存储为一个BinaryQuadraticModel对象。这个对象存储了数学表达式的线性和二次系数、任何常数项或偏移量，以及用于构建模型的变量类型。在这种情况下，我们构建的bqmmodel对象会显示以下内容：

以下示例演示了如何为一个QUBO问题减少一个Ising模型：

In [3]:
# 案例：QUBO 问题

bqmodel = dimod.BinaryQuadraticModel({'B': -1, 'A': -1}, {('A', 'B'): 2}, 0.0, dimod.BINARY)  # QUBO
model = dynex.BQM(bqmodel);


[DYNEX] MODEL CONVERTED TO QUBO
[DYNEX] PRECISION SET TO 0.0001
[DYNEX] QUBO: Constant offset of the binary quadratic model: 0.0


In [4]:
# 精确求解器
response = dimod.ExactSolver().sample(bqmodel)
print(response.first)

Sample(sample={'A': 1, 'B': 0}, energy=-1.0, num_occurrences=1)


## 与Dynex采样器互动

为了找到QM的最小能量状态（即为我们的QM提供最小能量值的变量值分配），Dynex SDK提供了采样器和求解器。求解器是运行问题的资源。采样器是多次运行问题以获取样本集合的过程，每个样本都是我们问题的一个可能解决方案。为方便起见，通常我们将总称Dynex的采样器，也包括求解器。

In [5]:
sampler = dynex.DynexSampler(model, mainnet=False);

[DYNEX] SAMPLER INITIALISED


一旦我们在程序中建立了我们的采样器，我们可以调用它来对我们的QM进行采样。每种类型的QM模型都有其自己的与采样器交互的方法，无论是QUBO、BinaryQuadrticModel，还是其他任何QM。我们使用Dynex的采样函数之一来调用采样器来对我们的QM进行采样，具体取决于我们使用的QM类型。例如，下面的代码片段演示了如何使用Dynex平台对名为bqm的BinaryQuadraticModel对象进行采样。

In [7]:
sampleset = sampler.sample(num_reads=32, annealing_time = 100);
print(sampleset)

[DYNEX|TESTNET] *** WAITING FOR READS ***
╭────────────┬───────────┬───────────────────────────┬─────────┬─────┬─────────┬───────┬─────┬──────────┬─────╮
│ DYNEXJOB   │ ELAPSED   │ WORKERS                   │ CHIPS   │ ✔   │ STEPS   │ LOC   │ ✔   │ ENERGY   │ ✔   │
├────────────┼───────────┼───────────────────────────┼─────────┼─────┼─────────┼───────┼─────┼──────────┼─────┤
│            │           │ *** WAITING FOR READS *** │         │     │         │       │     │          │     │
╰────────────┴───────────┴───────────────────────────┴─────────┴─────┴─────────┴───────┴─────┴──────────┴─────╯

[DYNEX] FINISHED READ AFTER 0.00 SECONDS
[DYNEX] SAMPLESET LOADED
   A  B energy num_oc.
0  1  0   -1.0       1
['BINARY', 1 rows, 1 samples, 2 variables]


在我们对QM进行采样后，采样器会返回一个SampleSet对象。这个对象包含了所有返回的样本及其相应的能量值、芯片数量、积分步数等信息。附加信息的内容取决于使用的采样器。随着用户对Dynex SDK和各种可用的采样器变得更加熟悉，花一些时间来探索SampleSet对象中提供的丰富信息通常是有用的。我们可以访问的SampleSet对象的一些关键属性和方法如下：

In [8]:
print(sampleset.first) # 能量最低的样本

Sample(sample={'A': 1, 'B': 0}, energy=-1.0, num_occurrences=1)


The sample shows the corresponding energy values for our QM and additional information like total energy, number of chips or number of integration steps.

In [9]:
sampleset # 全部样本

SampleSet(rec.array([([1, 0], -1., 1)],
          dtype=[('sample', 'i1', (2,)), ('energy', '<f8'), ('num_occurrences', '<i8')]), Variables(['A', 'B']), {}, 'BINARY')

每行显示了一个返回的样本（解决方案），以及相应的能量值、使用的芯片数量（每个Dynex芯片提供一次读取），和积分步数（=退火时间）。请注意，实际的积分步数可能会低于指定的退火时间，因为ODE积分在找到解决方案后结束。